<a href="https://colab.research.google.com/github/pratapwarrohit13/NLP/blob/main/Sentiment_Classification_using_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ktrain

     |████████████████████████████████| 25.3MB 109kB/s 
     |████████████████████████████████| 6.8MB 48.2MB/s 
     |████████████████████████████████| 983kB 55.0MB/s 
     |████████████████████████████████| 266kB 52.5MB/s 
     |████████████████████████████████| 1.9MB 50.8MB/s 
     |████████████████████████████████| 1.2MB 47.0MB/s 
     |████████████████████████████████| 471kB 48.5MB/s 
     |████████████████████████████████| 870kB 46.5MB/s 
     |████████████████████████████████| 3.3MB 29.8MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=a75dddacc81a17b5c8650c928e6f04c4ab1f5b68059247e6d3cfffeae958e5e2
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=52c11f1ba52bdbea38928de06729ab568fea4dcaae6d50ba703e383069776b14
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [2]:
!git clone  https://github.com/pratapwarrohit13/NLP.git

Cloning into 'NLP'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [4]:
df_train = pd.read_excel('/content/NLP/train.xlsx',dtype=str)
df_train

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg
...,...,...
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg


In [5]:
df_test = pd.read_excel('/content/NLP/test.xlsx',dtype=str)
df_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [6]:
(features_train,features_test), (target_train,target_test), preprocess = text.texts_from_df(train_df=df_train,text_column='Reviews',label_columns='Sentiment',val_df=df_test,
                   maxlen=400,preprocess_mode='bert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [7]:
features_train[0].shape

(25000, 400)

In [8]:
model = text.text_classifier(name='bert',
                             train_data = (features_train,features_test),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 400
done.


In [9]:
# Get Learning Rate
learner = ktrain.get_learner(model=model,
                             train_data = (features_train,features_test),
                             val_data = (target_train,target_test),
                             batch_size=6)

In [10]:
# This may take some days to complete
#learner.lr_find()
#learner.lr_plot()

# Optimal learning rate for this model is 2e-5


In [11]:
learner.fit_onecycle(lr=2e-5,epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - ETA: 0s - loss: 0.3588 - accuracy: 0.8318

KeyboardInterrupt: ignored

In [12]:
predictor = ktrain.get_predictor(learner.model,preproc=preprocess)
data = ['this movie was horrible. the plot was really boring and acting was okay. though ',
        'the filem really sucked. there is no plot and acting was bad',
        'what a beautiful movie.great plot,great acting, will see it again']
predictor.predict(data)


['neg', 'neg', 'pos']

In [13]:
predictor.save('/content/NLP/bert')